In [1]:
import tkinter as tk
from tkinter import messagebox
import cv2
import os
import pickle
from PIL import Image
import numpy as np

# Create the main application window
window = tk.Tk()
window.title("Face Recognition System")

# Function to create labels and entry fields
def create_widgets():
    tk.Label(window, text="Name", font=("Algerian", 20)).grid(column=0, row=0)
    tk.Label(window, text="Age", font=("Algerian", 20)).grid(column=0, row=1)
    tk.Label(window, text="Address", font=("Algerian", 20)).grid(column=0, row=2)

    global name_entry, age_entry, address_entry
    name_entry = tk.Entry(window, width=50, bd=5)
    age_entry = tk.Entry(window, width=50, bd=5)
    address_entry = tk.Entry(window, width=50, bd=5)

    name_entry.grid(column=1, row=0)
    age_entry.grid(column=1, row=1)
    address_entry.grid(column=1, row=2)

    tk.Button(window, text="Training", font=("Algerian", 20), bg='orange', fg='black', command=train_classifier).grid(column=0, row=4)
    tk.Button(window, text="Detect Face", font=("Algerian", 20), bg='orange', fg='black', command=detect_face).grid(column=1, row=4)
    tk.Button(window, text="Generate Dataset", font=("Algerian", 20), bg='orange', fg='black', command=generate_dataset).grid(column=2, row=4)

# Function to train the classifier and save labels
def train_classifier():
    data_dir = "path_to_your_data"
    current_id = 0
    label_ids = {}
    x_train = []
    y_labels = []

    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if file.endswith("png") or file.endswith("jpg"):
                path = os.path.join(root, file)
                label = os.path.basename(root).replace(" ", "-").lower()  # Folder name as label
                if label not in label_ids:
                    label_ids[label] = current_id
                    current_id += 1
                id_ = label_ids[label]
                pil_image = Image.open(path).convert("L")  # Convert to grayscale
                image_array = np.array(pil_image, "uint8")
                x_train.append(image_array)
                y_labels.append(id_)

    # Save the label names in a pickle file
    with open("labels.pickle", "wb") as f:
        pickle.dump(label_ids, f)

    x_train = np.array(x_train)
    y_labels = np.array(y_labels)

    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(x_train, y_labels)
    clf.write("classifier.xml")
    messagebox.showinfo('Result', 'Training data is completed!')

# Function to detect faces using the camera
def detect_face():
    faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.read("classifier.xml")

    # Load the labels from the pickle file
    with open("labels.pickle", "rb") as f:
        labels = pickle.load(f)
        labels = {v:k for k,v in labels.items()}  # Reverse to get names by ID

    # Open the default camera (0), use an IP address if needed
    url='webcam ip address'
    video_capture = cv2.VideoCapture(url)

    if not video_capture.isOpened():
        messagebox.showerror('Error', 'Camera failed to open')
        return

    while True:
        ret, img = video_capture.read()
        if not ret:
            messagebox.showerror('Error', 'Failed to capture frame')
            break

        gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = faceCascade.detectMultiScale(gray_image, 1.1, 10)
        
        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x+w, y+h), (255, 255, 255), 2)
            id_, pred = clf.predict(gray_image[y:y+h, x:x+w])
            confidence = int(100 * (1 - pred / 300))

            if confidence > 75:
                name = labels[id_]
                cv2.putText(img, name, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1, cv2.LINE_AA)
            else:
                cv2.putText(img, "UNKNOWN", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 1, cv2.LINE_AA)

        cv2.imshow("Face Detection", img)

        if cv2.waitKey(1) & 0xFF == 27:  # Exit on 'Esc' key
            break

    video_capture.release()
    cv2.destroyAllWindows()

# Function to generate a dataset of faces
def generate_dataset():
    if name_entry.get() == "" or age_entry.get() == "" or address_entry.get() == "":
        messagebox.showinfo('Result', 'Fill all three boxes')
        return
    
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        if len(faces) == 0:
            return None
        for (x, y, w, h) in faces:
            return img[y:y+h, x:x+w]

    url='ip_addressof webcam'
    cap = cv2.VideoCapture(url)  # Use 0 for the built-in camera

    img_id = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            messagebox.showerror('Error', 'Failed to capture frame')
            break
            
        face = face_cropped(frame)
        if face is not None:
            img_id += 1
            face = cv2.resize(face, (200, 200))
            face_gray = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

            file_name_path = f"data/user.{img_id}.jpg"
            cv2.imwrite(file_name_path, face_gray)
            cv2.putText(face_gray, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow("Cropped Face", face_gray)

            if cv2.waitKey(1) == 13 or img_id >= 200:  # Stop on Enter key or after 200 images
                break

    cap.release()
    cv2.destroyAllWindows()
    messagebox.showinfo('Result', 'Generating dataset is completed!')

# Initialize the application window
create_widgets()
window.geometry("800x200")
window.mainloop()
